In [ ]:
#This program was used to append the CSVs of tweets filtered by location and by tag and do some other house-keeping tasks.

In [ ]:
import csv
import json
import pandas as pd
import numpy as np
import re
import nltk
import time
import matplotlib.pyplot as plt
from sys import argv
from pprint import pprint
from ast import literal_eval
from langdetect import detect_langs
from langdetect import DetectorFactory
DetectorFactory.seed=0

import fun_2

In [ ]:
start_t=time.time()

count=0
A_id_str=pd.read_csv('tweetsByLoc.csv',usecols=['id_str'], engine='python',na_values={'None','NONE'}).fillna(0).astype(np.int64)
B_id_str=pd.read_csv('tweetsByTag.csv',usecols=['id_str'], engine='python',na_values={'None','NONE'}).fillna(0).astype(np.int64)

a=pd.read_csv('tweetsByLoc.csv',dtype={'id':float}, engine='python')
a['id_str']=A_id_str
b=pd.read_csv('tweetsByTag.csv',dtype={'id':float},engine='python')
b['id_str']=B_id_str

a=a.fillna('')#replace nan with blank string (so that each field can be used as string by default without having to use str().
b=b.fillna('')

In [ ]:
#print(a)
#print(b)
a=a.append(b,ignore_index=True)
#print(a)
a['id']=pd.to_numeric(a['id'],errors='coerce')
a=a.sort_values('id_str',)
a=a.reset_index(drop=True)

#print(a['id_str'],a['user_screen_name'])

In [ ]:
x=a.apply(fun_2.fun_lang2, axis=1)#since dataframe 'a' contains both "filtered by language" and "filtered by tag" tweets, 
                                  #so latter won't contain language information, so its language information is computed


logic=list(map( lambda arg: arg[0], x ))
trn=list(map( lambda arg: arg[1], x ))
logic_=list(map( lambda arg: not arg, logic ))
#list_count=list(map(lambda arg: arg[2],x))

c=a.loc[(a.index[logic])]
c['translate']=list(filter( None.__ne__,trn))
d=a.loc[(a.index[logic_])]



c['pros_text2']=c.apply(fun_2.f_clean2,axis=1)
d['pros_text2']=d.apply(fun_2.f_clean2,axis=1)
#In "pros_text" field, hashtag string was dropped along with '#'. Although, hashtag string is useful, so here new
#field "pros_text2" was created which contains hashtag string (without '#'). Like in "pros_text", here also we
#have removed mentions and urls. Whitespace compression has also been done.


y=c.apply(lambda arg: True if arg['translate']==True else False , axis=1) 

df_trans=c.loc[c.index[y]]#all data that needs translation 
df_non_trans=c.loc[c.index[~y]]#all data that doesn't needs translation 


df_non_trans.to_csv('tweetsNonTranslation.csv',index=None,header=True)
df_trans.to_csv('tweetsTranslation.csv',index=None,header=True)
#d.to_csv('rejectedTweets.csv',index=None,header=True)